In [2]:
#pip install --no-cache-dir -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
%%writefile ws_verAI_training.py
# Databricks notebook source
from all_imports import *

# COMMAND ----------

#set variables
# Open the file and load the file
with open('variables_setting.yaml') as f:
    variables = yaml.load(f, Loader=SafeLoader)
    print(variables)

globals().update(variables)


#os.environ["GCLOUD_PROJECT"] = "apmena-sandbox-dna-apac-dv"


client = bigquery.Client() 
sql = """
select *
from apmena-sandbox-dna-apac-dv.dbx_poc.sample_records
"""
df = client.query(sql).to_dataframe()

df.head()

split_date = df.sellout_date.max() + relativedelta(months=-3)
df_target = df[df['sellout_date'] > split_date]
df_target = df_target[[cid, sales]].groupby(cid).sum().reset_index()
df_target.columns = [cid, 'target_sales']
df_target['target_sales'] = df_target['target_sales'].apply(pd.to_numeric)

df_train = df[df['sellout_date'] <= split_date]

df_consumer = df[['consumer_code','tourist_flag', 'birthyear', 'gender', 'is_email_valid', 'is_address_valid', 'is_mobile_valid','is_instagram_contactable']].drop_duplicates() 
client = bigquery.Client() 
product_sql = """
select *
from apmena-sandbox-dna-apac-dv.dbx_poc_eu.compass_product
"""
df_product = client.query(product_sql).to_dataframe()

df_product.head()
merged_df = pd.merge(df_train, df_product, on='sap_product_code', how='left')

#get the unique consumer code to start building feature table
df_train = merged_df[[cid]].drop_duplicates()
df_train.shape
df_train = FE_pipeline.first_last_order(df_train, merged_df, cid, sellout_date)

merged_df['year_month'] = merged_df['sellout_date'].dt.to_period('M').astype(str)

df_train = FE_pipeline.Flag_purchasing_times_by_T(df_train,merged_df, Time_G, cid, sales)

df_train = FE_pipeline.get_lag_sales(df_train, merged_df, sellout_date, month_list, cid, sales)

df_train = FE_pipeline.past_xx_month_xx_features(df_train, merged_df, df_product, cid, sellout_date, sales, feature_dic, month_num = 24, column_name = 'signature_code')

df_train = FE_pipeline.feature_engineering_order_info(df_train, merged_df, cid, sellout_date, units, order_number, month_num = 12)
df_1 = FE_pipeline.get_recency(df_train, merged_df, cid, sellout_date, recency_target_cols)

df_cn = consumer_trans.consumer_trans(df_consumer)

df_2 = df_1.merge(df_cn, 'left', on = cid)
print(df_2.shape)
#training dataset post processing
df_2 = df_2[df_2['age']<100]
#change column types
object_cols = df_2.select_dtypes(include = 'object').columns.drop(['consumer_code','the_signature_code_with_the_maximum_of_sellout_gross_value_w_tax_p24m']).tolist()
float_cols = df_2.select_dtypes(include = 'float').columns.tolist()
print(object_cols)
print(float_cols)
for i in float_cols:
    df_2[i] = df_2[i].fillna(0)
    
df_2.columns = df_2.columns.str.replace(' ', '_')
df_2 = df_2.drop('is_instagram_contactable', axis=1)
df_2['consumer_id'] = df_2['consumer_code']



new_names = {
    "sum_of_sellout_gross_value_w_tax_of_H._Rubinstein_p24m":"sum_of_sales_of_HR_p24m",
    "sum_of_sellout_gross_value_w_tax_of_Kiehl's_p24m":"sum_of_sale_of_KS_p24m",
    'sum_of_sellout_gross_value_w_tax_of_Biotherm_p24m':"sum_of_sale_of_Bio_p24m",
    'sum_of_sellout_gross_value_w_tax_of_Yves_Saint_Laurent_p24m':"sum_of_sale_of_YSL_p24m",
    'sum_of_sellout_gross_value_w_tax_of_Lancome_p24m':"sum_of_sale_of_LC_p24m",
    'sum_of_sellout_gross_value_w_tax_of_Armani_p24m':"sum_of_sale_of_Armani_p24m",
    'sum_of_sellout_gross_value_w_tax_of_Shu_Uemura_p24m':"sum_of_sale_of_SU_p24m",
    'sum_of_sellout_gross_value_w_tax_of_Atelier_Cologne_p24m':"sum_of_sale_of_AC_p24m",
    'sum_of_sellout_gross_value_w_tax_of_Urban_Decay_p24m':"sum_of_sale_of_UD_p24m",
    'the_signature_code_with_the_maximum_of_sellout_gross_value_w_tax_p24m':'fav_brand_p24m',
    'max_of_sellout_gross_value_w_tax_by_signature_code_p24m':'maxsales_by_brand_p24m',
    "H._Rubinstein_recency": "HR_recency",
    "Kiehl's_recency":'KS_recency'
}


df_2 = df_2.rename(columns=new_names)
df_2.columns = df_2.columns.str.lower()
df_2.columns
training_df = pd.merge(df_2, df_target, on= 'consumer_code', how = 'left')
training_df['target_sales'] = training_df['target_sales'].fillna(0)

#writing data to bigquery
try:
    training_df.to_gbq(destination_table='dbx_poc.training_df', project_id= 'apmena-sandbox-dna-apac-dv')
except:
    skip

y = training_df['target_sales']
X = training_df.drop(['consumer_id','consumer_code','target_sales','first_order_date','last_order_date','fav_brand_p24m'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = lgb.LGBMRegressor(objective = 'tweedie', learning_rate=0.1, max_depth=4, reg_lambda=0.5, importance_type = 'gain').fit(X_train, y_train)

#write model to gcs bucket
import joblib
joblib.dump(model, 'lgb_bpp_kr_Luxe231130.pkl')

#get evaluation metrics
y_train_pred = model.predict(X_train)
y_train_true = y_train.to_numpy()
train_MAE, train_MSE, train_MAPE, train_WAPE, train_total_gap, train_spearman = em.print_regressin_metrics(y_train_true, y_train_pred)

y_test_true= y_test.to_numpy()
y_test_pred = model.predict(X_test)
test_MAE, test_MSE, test_MAPE, test_WAPE, test_total_gap, test_spearman = em.print_regressin_metrics(y_test_true, y_test_pred)

Overwriting ws_verAI_training.py


In [1]:
%%writefile Dockerfile
FROM python:3.9
WORKDIR /JIAYI-TEST
COPY requirements.txt .
COPY all_imports.py .
COPY variables_setting.yaml .
COPY steps ./steps

# Install dependencies with specified versions
RUN pip install --no-cache-dir -r requirements.txt

COPY ws_verAI_training.py ./ws_verAI_training.py

ENTRYPOINT ["python3", "ws_verAI_training.py"]

Overwriting Dockerfile


In [2]:
%%writefile cloudbuild.yaml

steps:
# Build the container image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/apmena-sandbox-dna-apac-dv/ws-vertexai-training', '.']
# Push the container image to Container Registry
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', 'gcr.io/apmena-sandbox-dna-apac-dv/ws-vertexai-training']
images:
- gcr.io/apmena-sandbox-dna-apac-dv/ws-vertexai-training

Overwriting cloudbuild.yaml


In [3]:
!gcloud builds submit --config cloudbuild.yaml

Creating temporary tarball archive of 26 file(s) totalling 361.1 KiB before compression.
Uploading tarball of [.] to [gs://apmena-sandbox-dna-apac-dv_cloudbuild/source/1701349806.283838-e74eb5bf216743c4a8a4e1165a76676e.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/apmena-sandbox-dna-apac-dv/locations/global/builds/015a857a-065d-46a3-bb6e-60758ef4144b].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/015a857a-065d-46a3-bb6e-60758ef4144b?project=1022207864171 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "015a857a-065d-46a3-bb6e-60758ef4144b"

FETCHSOURCE
Fetching storage object: gs://apmena-sandbox-dna-apac-dv_cloudbuild/source/1701349806.283838-e74eb5bf216743c4a8a4e1165a76676e.tgz#1701349807398744
Copying gs://apmena-sandbox-dna-apac-dv_cloudbuild/source/1701349806.283838-e74eb5bf216743c4a8a4e1165a76676e.tgz#1701349807398744...
/ [1 files][ 61.9 KiB/ 61.9 KiB]                                 

In [4]:
from datetime import datetime
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
JOB_NAME=f"lgb_{TIMESTAMP}"
CUSTOM_CONTAINER_IMAGE_URI="gcr.io/apmena-sandbox-dna-apac-dv/ws-vertexai-training:latest"
REGION="asia-east2"

print(TIMESTAMP)

20231130131341


In [5]:
%%writefile config.yaml
workerPoolSpecs:
  machineSpec:
    machineType: n1-standard-8
  replicaCount: 1
  containerSpec:
    imageUri: gcr.io/apmena-sandbox-dna-apac-dv/ws-vertexai-training:latest

Overwriting config.yaml


In [6]:
!gcloud ai custom-jobs create \
  --region={REGION} \
  --display-name={JOB_NAME} \
  --config=config.yaml

Using endpoint [https://asia-east2-aiplatform.googleapis.com/]
CustomJob [projects/1022207864171/locations/asia-east2/customJobs/1238116063573442560] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai custom-jobs describe projects/1022207864171/locations/asia-east2/customJobs/1238116063573442560

or continue streaming the logs with the command

  $ gcloud ai custom-jobs stream-logs projects/1022207864171/locations/asia-east2/customJobs/1238116063573442560


In [14]:
project_id = 'apmena-sandbox-dna-apac-dv' 
gcs_bucket = 'gs://dbx_poc'
location = 'asia-east2'

In [8]:
from google.cloud import aiplatform as vertexai

In [9]:
vertexai.init(project=project_id,staging_bucket=gcs_bucket)

In [10]:
worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-standard-8"
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": CUSTOM_CONTAINER_IMAGE_URI
            },
        }
    ]
print(worker_pool_specs)

[{'machine_spec': {'machine_type': 'n1-standard-8'}, 'replica_count': 1, 'container_spec': {'image_uri': 'gcr.io/apmena-sandbox-dna-apac-dv/ws-vertexai-training:latest'}}]


In [11]:
job = vertexai.CustomJob(
    display_name=f"{JOB_NAME}",
    worker_pool_specs=worker_pool_specs
)


In [12]:
job.run(
    sync=False,
)

Creating CustomJob


In [2]:
%%writefile only_model_training.py
import pandas as pd
import numpy as np
import db_dtypes


from google.cloud import aiplatform
import vertexai
from google.cloud import bigquery

import joblib

import sklearn
from sklearn.model_selection import train_test_split
import lightgbm as lgb

client = bigquery.Client() 
sql = """
select *
from apmena-sandbox-dna-apac-dv.dbx_poc.training_df
"""

training_df = client.query(sql).to_dataframe()
y = training_df['target_sales']
X = training_df.drop(['consumer_id','consumer_code','target_sales','first_order_date','last_order_date','fav_brand_p24m'], axis = 1)
X_train, X_t
est, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = lgb.LGBMRegressor(objective = 'tweedie', learning_rate=0.1, max_depth=4, reg_lambda=0.5, importance_type = 'gain').fit(X_train, y_train)

#write model to gcs bucket
import joblib
joblib.dump(model, 'lgb_bpp_kr_Luxe231130.pkl')


Writing only_model_training.py


In [3]:
def create_experiment_sample(
    experiment_name: str,
    experiment_description: str,
    project: str,
    location: str,
):
    aiplatform.init(
        experiment=experiment_name,
        experiment_description=experiment_description,
        project=project,
        location=location,
    )



In [5]:
create_experiment_sample(experiment_description='This is a test for using experiment',
experiment_name = 'lgbtest',
project = project_id,
location = location)

NameError: name 'project_id' is not defined

In [25]:
def create_experiment_run_sample(
    experiment_name: str,
    run_name: str,
    project: str,
    location: str,
):
    aiplatform.init(experiment=experiment_name, project=project, location=location)

    aiplatform.start_run(run=run_name)


In [28]:
create_experiment_run_sample('lgbtest','trainingv2',project_id, 'asia-east2')

Associating projects/1022207864171/locations/asia-east2/metadataStores/default/contexts/lgbtest-trainingv2 to Experiment: lgbtest


In [ ]:
with aiplatform.start_run('training') as my_run:

    y = training_df['target_sales']
    X = training_df.drop(['consumer_id','consumer_code','target_sales','first_order_date','last_order_date','fav_brand_p24m'], axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #write model to gcs bucket
    import joblib
    joblib.dump(model, 'lgb_bpp_kr_Luxe231130.pkl')
    
    #take the trained model hp
    my_run.log_params({'learning_rate':0.1, 'max_depth':4, 'reg_lambda':0.5, 'importance_type':'gain'})
    model = lgb.LGBMRegressor(objective = 'tweedie', learning_rate=0.1, max_depth=4, reg_lambda=0.5, importance_type = 'gain').fit(X_train, y_train)

    aiplatform.log_model(
    model=model,
    input_example=X.head(5),
    display_name='lgb_bpp_kr_Luxe231130',
)

In [21]:
from google.cloud import aiplatform

In [29]:
def create_custom_job_with_experiment_autologging_sample(
    project: str,
    location: str,
    staging_bucket: str,
    display_name: str,
    script_path: str,
    container_uri: str,
    service_account: str,
    experiment: str,
    experiment_run: str) -> None:
    aiplatform.init(project=project, location=location, staging_bucket=staging_bucket, experiment=experiment)

    job = aiplatform.CustomJob.from_local_script(
        display_name=display_name,
        script_path=script_path,
        container_uri=container_uri,
        enable_autolog=True,
    )

    job.run(
        service_account=service_account,
        experiment=experiment,
        experiment_run = experiment_run)

In [30]:
create_custom_job_with_experiment_autologging_sample(
    project_id,
    location,
    staging_bucket=gcs_bucket,
    display_name='training job of lgb model',
    script_path='./ws_verAI_training.py',
    container_uri= 'asia-docker.pkg.dev/vertex-ai/training/sklearn-cpu.1-0:latest',
    service_account = '1022207864171-compute@developer.gserviceaccount.com',
    experiment = 'lgbtest',
    experiment_run = 'trainingv2')

Training script copied to:
gs://dbx_poc/aiplatform-2023-11-30-13:46:15.317-aiplatform_custom_trainer_script-0.1.tar.gz.
Creating CustomJob
CustomJob created. Resource name: projects/1022207864171/locations/asia-east2/customJobs/9101401012962328576
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1022207864171/locations/asia-east2/customJobs/9101401012962328576')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/asia-east2/training/9101401012962328576?project=1022207864171
CustomJob projects/1022207864171/locations/asia-east2/customJobs/9101401012962328576 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1022207864171/locations/asia-east2/customJobs/9101401012962328576 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1022207864171/locations/asia-east2/customJobs/9101401012962328576 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1022207864171/locations/asia-east2/customJobs/91014010129623

RuntimeError: Job failed with:
code: 3
message: "The replica workerpool0-0 exited with a non-zero status of 1. \nTraceback (most recent call last):\n  File \"/opt/conda/lib/python3.10/runpy.py\", line 196, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n  File \"/opt/conda/lib/python3.10/runpy.py\", line 86, in _run_code\n    exec(code, run_globals)\n  File \"/root/.local/lib/python3.10/site-packages/aiplatform_custom_trainer_script/task.py\", line 7, in <module>\n    from all_imports import *\nModuleNotFoundError: No module named \'all_imports\'\n\nTo find out more about why your job exited please check the logs: https://console.cloud.google.com/logs/viewer?project=1022207864171&resource=ml_job%2Fjob_id%2F9101401012962328576&advancedFilter=resource.type%3D%22ml_job%22%0Aresource.labels.job_id%3D%229101401012962328576%22"
